<a href="https://colab.research.google.com/github/Vedal987/nybble-gym/blob/main/train-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Nybble on the Cloud

Github Repo: [https://github.com/Vedal987/nybble-gym](https://github.com/Vedal987/nybble-gym)

Pybullet source code: https://github.com/bulletphysics/bullet3/tree/master/examples/pybullet/

## Install Dependencies and Stable Baselines Using Pip


```
pip install stable-baselines3[extra]
```

In [ ]:
!pip install stable-baselines3[extra] pybullet

!wget --no-cache --backups=1 "https://raw.githubusercontent.com/Vedal987/nybble-gym/main/opencat_gym_env_nybble.py"
!mkdir models
!wget --no-cache --backups=1 -P models "https://raw.githubusercontent.com/Vedal987/nybble-gym/main/models/nybble.urdf"

## Import policy, RL agent, Wrappers

In [ ]:
import os 

from stable_baselines3 import SAC
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
from opencat_gym_env_nybble import OpenCatGymEnv

## Create and wrap the environment with `VecNormalize`

Normalizing input features may be essential to successful training of an RL agent (by default, images are scaled but not other types of input), for instance when training on [PyBullet](https://github.com/bulletphysics/bullet3/) environments. For that, a wrapper exists and will compute a running average and standard deviation of input features (it can do the same for rewards).

More information about `VecNormalize`:
- [Documentation](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#stable_baselines3.common.vec_env.VecNormalize)
- [Discussion](https://github.com/hill-a/stable-baselines/issues/698)

In [ ]:
env = OpenCatGymEnv(False)
env = make_vec_env(lambda: env, n_envs=1) # 25 for PPO
env = VecNormalize(env, training=True, norm_obs=True, norm_reward=True) # This might be necessary or break things

### Train the agent

In [ ]:
model = SAC("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=100000)

### Save the agent and the normalization

In [ ]:
# Don't forget to save the VecNormalize statistics when saving the agent
log_dir = "/tmp/"
model.save(log_dir + "sac_opencat")
stats_path = os.path.join(log_dir, "vec_normalize.pkl")
env.save(stats_path)

### Test model: load the saved agent and normalization

In [ ]:
# Load the agent
model = SAC.load(log_dir + "sac_opencat")

# Load the saved statistics
env = OpenCatGymEnv()
env = make_vec_env(lambda: env, n_envs=1) # 25 for PPO
env = VecNormalize(env, norm_obs=True) # This might be necessary or break things

#  do not update them at test time
env.training = False
# reward normalization is not needed at test time
env.norm_reward = False

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
mean_reward, std_reward = evaluate_policy(model, env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")